In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

In [3]:
chrome_path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(executable_path = chrome_path)

categories = ['moisturizing-cream-oils-mists',
             'cleanser',
             'facial-treatments',
             'eye-treatment-dark-circle-treatment',
             'facial-treatment-masks',
             'sunscreen-sun-protection',
             'lip-treatments']

final_df = pd.DataFrame(columns = ['category', 'URL'])

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 87
Current browser version is 89.0.4389.90 with binary path /Applications/Google Chrome.app/Contents/MacOS/Google Chrome


In [48]:
for category in categories:

    page_num = 1 

    while True:
        url = 'https://www.sephora.com/shop/'+ category + '?pageSize=300&currentPage=' + str(page_num)
        driver.get(url)
        time.sleep(20)
        try: #check to see if the page is empty   
            if driver.find_element_by_class_name('css-3a7b4s').is_displayed():
                break
            
        except:
            try:
                xpath = '//*[@id="modalDialog"]/button'
                btn = driver.find_element_by_xpath(xpath)
                btn.click()
                time.sleep(20)
            except:
                pass
            old_len = 0
            while True:
                browser = scrollDown(driver, 20) 
                time.sleep(10) 
                slug = driver.find_elements_by_class_name('css-ix8km1') 
                new_len = len(slug)
                if old_len == new_len: 
                    break
                else:
                    old_len = new_len
            slugURL = []
            for a in slug:
                subURL = {}
                subURL['category'] = category 
                subURL['URL'] = a.get_attribute('href')
                slugURL.append(subURL)
            df = pd.DataFrame(slugURL)
            page_num += 1
            final_df = pd.concat([final_df, df], axis = 0, ignore_index = True) 
driver.close()

KeyboardInterrupt: 

In [51]:
final_df.to_csv('moisturize.csv')